#### Objetivo

O objetivo deste trabalho é realizar o cálculo de custo de matéria prima para todos os códigos de peças contidos em uma planilha de excel, seja o código referente à uma peça, conjunto soldado ou conjunto montado. 

#### Motivação

Esta ideia parte da necessidade de cálculo de custos de não conformidades. Dado que as peças dos projetos internos seguem um fluxo de entrada em fábrica diferente dos projetos realizados pela empresa terceirizada de engenharia. Isso faz com que os primeiros não estejam presentes na base de dados do de peças SteelProjects, software para programação e gestão dos projetos em fábrica. 

Com isso, toda não conformidade envolvendo peças de projetos internos requer que seu peso, matéria prima e dimensões sejam pesquisados manualmente e lançados em uma planilha de Excel para que o seu custo seja computado corretamente na planilha de custos de não conformidades. Este problema se agrava quando as não conformidades envolvem conjuntos soldados ou montados, onde o correto seria calcular o custo unitário de cada um de seus componentes e somar os custos. Porém, devido à grande quantidade de tempo necessária para fazê-lo, o custo da peça é calculado apenas multiplicando o peso total do conjunto por uma média de valor por quilograma de diversos produtos.

#### Metodologia

Para obter os dados das peças de projetos internos, será utilizado a lista do projeto gerada pela engenharia. Essa lista contém quase todas as informações necessárias para o calculo como se o código é uma peça única, conjunto soldado ou conjunto montado, matéria prima, dimensões, peso e código do ERP referente à matéria prima. Para obter o custo da MP a partir do código do ERP será necessário realizar uma busca em outra planilha extrai essas informações do ERP.

##### Ferramentas

Para este trabalho será utilizada a já conhecida biblioteca "pandas", já explicada em sala e a biblioteca "openpyxl" não lecionada em sala. A necessidade da segunda surge dado o fato que a separação entre conjunto soldado e montado é feita a partir da coloração da linha principal da conjunto. Isso é necessário pois a biblioteca openpyxl trabalha com propridades da planilha excel além dos dados, podendo coletar a cor das linhas, enquanto pandas não.

In [57]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [41]:
# Lendo a planilha com o openpyxl

workbook = load_workbook(r"C:\Users\guico\OneDrive\Documentos\OneDrive\UFPR\Matérias\8° Período\Mineração de Dados - Alexandre\Trabalho pyhton\P003.0005.A001_E00.xlsm")
ws = workbook["Lista de Materiais"]

In [42]:
# Salvando as linhas da planilha em uma lista

lista_ws = []
for row in ws.iter_rows(values_only=True):
    lista_ws.append(row)

print(lista_ws)


[(None, 'Nº DOCUMENTO:\nP003.0005.A001_E00', None, 'LISTA DE MATERIAIS\nPOSTE SLS 9M V045', None, None, None, None, None, None, None, None, 'Nº DA REVISÃO:\nREV-2024-00', None, None), ('MULTIPLICADOR:\n', 1, None, 'SITE:', None, None, None, None, None, 'REVISADO:\n', None, None, 'APROVADO:\n', 'EVERTOM W.', 'DATA:'), ('CÓDIGO', 'COMPONENTE', 'DESCRIÇÃO', 'CÓDIGO SANKHYA', 'CARACTERÍSTICAS', 'MATERIAL', 'ESP.', 'QDT. UNIT.', 'QDT. TOTAL', 'PESO KG', 'PESO TOTAL', 'COMPRIMENTO', 'LARGURA', 'TRATAMENTO', 'PINTURA'), (None, 'P003.0005.M001_E00', 'CHUMBADOR', None, 'MONTAGEM FINAL', 'N/A', 'N/A', 1, 1, 39, 39, 583, 583, 'N/A', 'N/A'), ('AR.LS.P19.05', None, 'FIXAÇÕES', 538, 'ARRUELA LISA 3/4"', 'ASTM-A325', 'N/A', 10, 10, 0, 0, 0, 0, 'GALV. A FOGO', 'N/A'), ('PO.SX.P19.05-10', None, 'FIXAÇÕES', 1838, 'PORCA SEXT. 3/4"-10', 'ASTM-A325', 'N/A', 30, 30, 0, 0, 0, 0, 'GALV. A FOGO', 'N/A'), ('  P003.0005.S010_E00', None, 'CHUMBADOR', None, 'SOLDAGEM', 'N/A', 'N/A', 1, 1, 37.2, 37.2, 583, 583, 'G

In [43]:
# Como a biblioteca não consegue acessar a cor de linhas ou colunas, apenas de células, realizo outro laço for para percorrer as células apenas da 
#   primeira coluna, o que me retorna a cor da linha

lista_cores = []
for cell in ws['A']:
    cor = cell.fill.fgColor.theme # Obter a cor tratada, os valores com texto são células sem cores
    lista_cores.append(cor) 
    
print(lista_cores) # Células que não estão pintadas não possuem .theme, por isso aparece este "erro"

lista_cores = lista_cores[3:] # Removendo as duas primeiras linhas e o cabeçalho
lista_cores = [0 if type(i) != int else i for i in lista_cores] # Transformando o aviso que aparece para células não pintadas (o aviso não é salvo como string e sim como um objeto do openpyxl)

print(lista_cores)

[Values must be of type <class 'int'>, Values must be of type <class 'int'>, 1, 8, Values must be of type <class 'int'>, Values must be of type <class 'int'>, 2, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, 8, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, 2, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int'>, Values must be of type <class 'int

In [44]:
# Tranformando a lista em df 

df = pd.DataFrame(lista_ws[2:])
df.columns = df.iloc[0]
df = df[1:]
df

,CÓDIGO,COMPONENTE,DESCRIÇÃO,CÓDIGO SANKHYA,CARACTERÍSTICAS,MATERIAL,ESP.,QDT. UNIT.,QDT. TOTAL,PESO KG,PESO TOTAL,COMPRIMENTO,LARGURA,TRATAMENTO,PINTURA
1,None,P003.0005.M001_E00,CHUMBADOR,None,MONTAGEM FINAL,N/A,N/A,1,1,39,39,583,583,N/A,N/A
2,AR.LS.P19.05,None,FIXAÇÕES,538,"ARRUELA LISA 3/4""",ASTM-A325,N/A,10,10,0,0,0,0,GALV. A FOGO,N/A
3,PO.SX.P19.05-10,None,FIXAÇÕES,1838,"PORCA SEXT. 3/4""-10",ASTM-A325,N/A,30,30,0,0,0,0,GALV. A FOGO,N/A
4,P003.0005.S010_E00,None,CHUMBADOR,None,SOLDAGEM,N/A,N/A,1,1,37.2,37.2,583,583,GALV. A FRIO,N/A
5,None,P003.0005.P001_E00,TALA CHUMBADOR,154,"CH 1/4""",CIVIL 300,6.35,20,20,1.09,21.8,277.22,80,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,0018.0001.M003_R00,None,UNIDADE,9031,BERÇO DE MADEIRA PARA POSTE MTOPO,MADEIRA,N/A,1,1,0,0,0,0,N/A,N/A
258,P003.0005.P223_E00,None,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1,0.06,0.06,207.5,35,N/A,N/A
259,P003.0005.P229_E00,None,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1,0.03,0.03,105,35,N/A,N/A
260,P003.0005.P533_E00,None,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,3,3,0.02,0.06,67.5,35,N/A,N/A


In [45]:
df["CÓDIGO"] = df["CÓDIGO"].str.strip() # Remove espaços no início e fim do texto
df["CÓDIGO"] = df["CÓDIGO"].str.replace(" ","",regex=False) # Remove espaços em branco no meio do texto (possíveis erros de digitação)
df["CÓDIGO"] = df["CÓDIGO"].fillna("") # Preenchendo células sem dados com vazio para o condicional da coluna ÉPeça? funcionar
df["COMPONENTE"] = df["COMPONENTE"].str.strip()
df["COMPONENTE"] = df["COMPONENTE"].str.replace(" ","",regex=False)
df["COMPONENTE"] = df["COMPONENTE"].fillna("")

In [ ]:
# Pareando linhas com a lista de cores
df["Cor"] = lista_cores 

In [ ]:
# Define o que é peça e o que não é

regex = r'^[A-Za-z0-9]{4}\.[A-Za-z0-9]{4}\.[A-Za-z0-9]{4}_[A-Za-z0-9]{3}$' # Estabelece o formato de texto que definirá um código de peça

df["ÉPeça?"] = df["CÓDIGO"].str.match(regex) | df["COMPONENTE"].str.match(regex) 

,CÓDIGO,COMPONENTE,DESCRIÇÃO,CÓDIGO SANKHYA,CARACTERÍSTICAS,MATERIAL,ESP.,QDT. UNIT.,QDT. TOTAL,PESO KG,PESO TOTAL,COMPRIMENTO,LARGURA,TRATAMENTO,PINTURA,Cor,ÉPeça?,CODPROD,Custo Médio com ICMS
0,,P003.0005.M001_E00,CHUMBADOR,None,MONTAGEM FINAL,N/A,N/A,1,1,39,39,583,583,N/A,N/A,8,True,NaN,NaN
1,AR.LS.P19.05,,FIXAÇÕES,538,"ARRUELA LISA 3/4""",ASTM-A325,N/A,10,10,0,0,0,0,GALV. A FOGO,N/A,0,False,538.0,7.36
2,AR.LS.P19.05,,FIXAÇÕES,538,"ARRUELA LISA 3/4""",ASTM-A325,N/A,10,10,0,0,0,0,GALV. A FOGO,N/A,0,False,538.0,7.36
3,PO.SX.P19.05-10,,FIXAÇÕES,1838,"PORCA SEXT. 3/4""-10",ASTM-A325,N/A,30,30,0,0,0,0,GALV. A FOGO,N/A,0,False,1838.0,1.92
4,PO.SX.P19.05-10,,FIXAÇÕES,1838,"PORCA SEXT. 3/4""-10",ASTM-A325,N/A,30,30,0,0,0,0,GALV. A FOGO,N/A,0,False,1838.0,1.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,P003.0005.P229_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1,0.03,0.03,105,35,N/A,N/A,0,True,1033.0,7.57
466,P003.0005.P533_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,3,3,0.02,0.06,67.5,35,N/A,N/A,0,True,1033.0,7.57
467,P003.0005.P533_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,3,3,0.02,0.06,67.5,35,N/A,N/A,0,True,1033.0,7.57
468,P003.0005.P543_E00,,INSUMOS,1033,TRILHO DIN,AISI 1010-GALV.,N/A,1,1,0.11,0.11,380,35,N/A,N/A,0,True,1033.0,7.57


In [ ]:
# Importando tabela de custos
df_custos = pd.read_excel(r"C:\Users\guico\OneDrive\Documentos\OneDrive\UFPR\Matérias\8° Período\Mineração de Dados - Alexandre\Trabalho pyhton\CustosSankhyaFinal.xlsx")

# Realizando a mesclagem de dados para puxar o custo unitário
df = df.merge(df_custos[["CODPROD","Custo Médio com ICMS"]], how = "left", left_on = "CÓDIGO SANKHYA", right_on = "CODPROD")
df["Custo Médio com ICMS"] = df["Custo Médio com ICMS"].fillna("")

In [60]:
# Calculando o custo de cada componente, se for peça multiplica o custo un pelo peso, se não, pela quantidade

df["Custo Total"] = np.where(
    df["ÉPeça?"] == False,  # Condição
    df["Custo Médio com ICMS"] * df["QDT. TOTAL"],
    df["Custo Médio com ICMS"] * df["PESO TOTAL"] 
)

In [ ]:
df[df["Cor"]!=0]

# Salvar cores em uma lista (apenas cores)
# Salvar o índice das linhas em outra lista
# Criar as regras de fatiamento (dentro do for?)
# Fatiar o df utlizando os índices das linhas com cor
# Somar valores da coluna custo
# Salvar isso em outro df

# Se der tempo criar uma planilha para armazenar isso

,CÓDIGO,COMPONENTE,DESCRIÇÃO,CÓDIGO SANKHYA,CARACTERÍSTICAS,MATERIAL,ESP.,QDT. UNIT.,QDT. TOTAL,PESO KG,PESO TOTAL,COMPRIMENTO,LARGURA,TRATAMENTO,PINTURA,Cor,ÉPeça?,CODPROD,Custo Médio com ICMS,Custo Total
0,,P003.0005.M001_E00,CHUMBADOR,None,MONTAGEM FINAL,N/A,N/A,1,1,39,39,583,583,N/A,N/A,8,True,NaN,NaN,NaN
5,P003.0005.S010_E00,,CHUMBADOR,None,SOLDAGEM,N/A,N/A,1,1,37.2,37.2,583,583,GALV. A FRIO,N/A,2,True,NaN,NaN,NaN
12,,P003.0005.M230_E00,MÓDULO 1 EQUIPAMENTOS,None,MONTAGEM FINAL,N/A,N/A,1,1,297.46,297.46,3655,800,N/A,N/A,8,True,NaN,NaN,NaN
29,P003.0005.S100_E00,,MÓDULO 1,None,SOLDAGEM,N/A,N/A,1,1,297.46,297.46,3655,800,GALV. A FOGO,RAL 6004,2,True,NaN,NaN,NaN
90,,P003.0005.M150_E00,PORTA,None,MONTAGEM,N/A,N/A,4,4,11,44,670,476,N/A,N/A,9,True,NaN,NaN,NaN
113,,P003.0005.M170_E00,BASE FILTRO,None,MONTAGEM,N/A,N/A,1,1,2.2,2.2,516,366,N/A,N/A,9,True,NaN,NaN,NaN
127,,P003.0005.M180_E00,BANDEJA EXAUSTOR,None,MONTAGEM,N/A,N/A,2,2,0.77,1.54,388.18,220,N/A,N/A,9,True,NaN,NaN,NaN
146,,P003.0005.M190_E00,MONTAGEM RACK/RRU,None,MONTAGEM,N/A,N/A,4,4,1.71,6.84,555.13,81.25,N/A,N/A,9,True,NaN,NaN,NaN
158,,P003.0005.M200_E00,TAMPA PRENSA CABOS,None,MONTAGEM,N/A,N/A,2,2,0.18,0.36,220,90,N/A,N/A,9,True,NaN,NaN,NaN
171,,P003.0005.M220_E00,PAINEL AC,None,MONTAGEM,N/A,N/A,1,1,4.09,4.09,480,105,N/A,N/A,9,True,NaN,NaN,NaN


In [64]:
df[df["Cor"]!=0].index

Int64Index([  0,   5,  12,  29,  90, 113, 127, 146, 158, 171, 197, 219, 237,
            260, 279, 306, 315, 325, 339, 370, 380, 390],
           dtype='int64')